# Laboratorio #4 - Análisis de sentimientos
* Oscar Juárez - 17315
* José Pablo Cifuentes - 17509
* Paul Belches - 17088

In [ ]:
import pandas as pd
import numpy as np
import math
import re

In [ ]:
df = pd.read_csv('./files/GrammarandProductReviews.csv')
print([var for var in df])

In [ ]:
clean_df = df[['brand', 'manufacturer', 'name', 'reviews.username', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.rating', 'reviews.text']].copy()
clean_df

In [ ]:
clean_reviews = []

for line in clean_df['reviews.text']:
    line = str(line)
    # Quitar saltos de linea y convertirlo a minusculas
    line = line.rstrip('\n').lower()
    # Quitar URLS
    line = re.sub(r'^https?:\/\/.[\r\n]', '', line)
    # Quitar el resto de expresiones regulares, excepto . ? ! y '
    line = re.sub(r"[^\w.?!\d'\s]", '', line)
    # Quitar números
    line = re.sub(' \d+', ' ', line)
    # Quitar espacios extra
    line = line.strip(' \t\n\r')
    # Reemplazar ! y ? por puntos.
    line = line.replace('!','.').replace('?','.')
    # Quitar multiples puntos por solo uno
    line = re.sub(r'\.+', ".", line)
    # Finalmente, quitamos apostrofes
    line = line.replace("'", '')

    clean_reviews.append(line)

### Validamos que no hayan usuarios o nombres de productos vacíos

Empezamos obteniendo la lista que contenga los índices de los valores vacíos. Luego, reemplazamos cada valor por '---'

In [ ]:
# Creamos la lista de índices con valores vacíos
usernames = [str(prod) for prod in clean_df['reviews.username'].tolist()]
empties = np.where(pd.isnull(clean_df[['reviews.username']]))[0].tolist()

for index in empties:
    usernames[index] = '---'

Ahora, hacemos lo mismo para los productos.

In [ ]:
products = [str(prod) for prod in clean_df['name'].tolist()]
empties = np.where(pd.isnull(clean_df[['reviews.username']]))[0].tolist()

for index in empties:
    products[index] = '---'

### Crear CSV con datos limpios

Primero, se cambian las columnas por aquellas que esten limpias.

In [ ]:
clean_df['reviews.text'] = clean_reviews
clean_df['reviews.username'] = usernames
clean_df['name'] = products

Luego, crear CSV limpio

In [ ]:
df = pd.DataFrame(clean_df)
df.to_csv('./files/CleanData.csv', index=False)

## Ejercicio 6



In [ ]:
clean_df = pd.read_csv('./files/CleanData.csv')

### Inciso 6.3
Cuáles son los usuarios que dan la mayor cantidad de reviews a distintos productos.

In [ ]:
# Diccionario de usuario -> producto
prod_user = clean_df[['name', 'reviews.username']].copy().values

Se crea un diccionario que tendrá al nombre del usuario como llave y la cantidad de productos comprados como valor.

In [ ]:
# Diccionario de todos los usuarios distintos. El valor será un arreglo de productos, sin repetición.
user_dict = {}

for prod, user in prod_user:

    # Si el usuario no está en el diccionario, agregarlo a él y el producto como valor.
    if user not in user_dict: 
        user_dict[user] = []
        user_dict[user].append(prod)

    # Si el usuario sí está en el diccionario y el producto no está en su lista de productos, agregarlo.
    elif prod not in user_dict[user]:
        user_dict[user].append(prod)

# Ahora, cambiar los productos comprados por la cantidad de productos.
for user, products in user_dict.items():
    user_dict[user] = len(products)

Se ordena el arreglo en base a los productos

In [ ]:
ordered = sorted(user_dict.items(), key=lambda x: x[1], reverse=True)

Se imprimen la primeras 7 posiciones

In [ ]:
print(ordered[0:7])

#### Respuesta:

Sin tomar en cuenta a los clientes "Anónimos":

1. Mike: 29 productos
2. Chris: 27 productos
3. Sandy: 23 productos
4. John: 23 productos
5. lisa: 22 productos